In [1]:
import chromadb
import logging
import polars as pl

In [2]:
from smart_procurement.embeeders import MixedbreadEmbedder
from smart_procurement.embeeding_funcs import MixedbreadEmbeddingFunction
from smart_procurement.models.chroma_cc_model import CommodityCodesListChromaDB

/Users/datapsycho/Library/Application Support/hatch/env/virtual/smart-procurement/Pzhnh7aS/smart-procurement/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [3]:
logging.basicConfig(
    format='%(asctime)s | %(levelname)s : %(message)s',
    level=logging.INFO,
    # stream=sys.stdout,
    force=True
)

In [4]:
EMB_MODEL_PATH = "../models/embeddings/mxbai-embed-large-v1"

In [5]:
emb_model = MixedbreadEmbedder(EMB_MODEL_PATH)

2024-08-24 16:51:57,769 | INFO : Use pytorch device_name: mps
2024-08-24 16:51:57,770 | INFO : Load pretrained SentenceTransformer: ../models/embeddings/mxbai-embed-large-v1
2024-08-24 16:52:00,954 | INFO : Model loaded from ../models/embeddings/mxbai-embed-large-v1


# Adding Cached Data

In [6]:
client = await chromadb.AsyncHttpClient(host="localhost", port=8011)

2024-08-24 16:52:04,518 | INFO : Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-08-24 16:52:04,553 | INFO : Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-08-24 16:52:04,593 | INFO : HTTP Request: GET http://localhost:8011/api/v1/tenants/default_tenant "HTTP/1.1 200 OK"
2024-08-24 16:52:04,598 | INFO : HTTP Request: GET http://localhost:8011/api/v1/databases/default_database?tenant=default_tenant "HTTP/1.1 200 OK"


In [10]:
# await client.delete_collection("test_collection")

In [8]:
test_collection = await client.get_collection(
    "test_collection", 
    embedding_function=MixedbreadEmbeddingFunction(
        embedder=MixedbreadEmbedder(EMB_MODEL_PATH)
    ),
    # metadata={
    # "hnsw:space": "cosine",
    # "hnsw:construction_ef": 100,
    # }
)

2024-08-24 16:52:43,040 | INFO : Use pytorch device_name: mps
2024-08-24 16:52:43,041 | INFO : Load pretrained SentenceTransformer: ../models/embeddings/mxbai-embed-large-v1
2024-08-24 16:52:45,778 | INFO : Model loaded from ../models/embeddings/mxbai-embed-large-v1
2024-08-24 16:52:45,829 | INFO : HTTP Request: GET http://localhost:8011/api/v1/collections/test_collection?tenant=default_tenant&database=default_database "HTTP/1.1 200 OK"


In [22]:
await test_collection.add(
    ids=["1", "2"],
    metadatas=[{"id": 1}, {"id": 2}],
    documents=["A nice Cat", "An amazing Truck"],
)

2024-08-18 16:19:29,443 | INFO : Generating embeddings for 2 sentences with 1024 dimensions.
Batches: 100%|██████████| 1/1 [00:00<00:00,  1.25it/s]
2024-08-18 16:19:30,284 | INFO : HTTP Request: GET http://localhost:8011/api/v1/pre-flight-checks "HTTP/1.1 200 OK"
2024-08-18 16:19:30,319 | INFO : HTTP Request: POST http://localhost:8011/api/v1/collections/f8c69627-b541-497c-9ba5-c48f01e31e59/add "HTTP/1.1 201 Created"


In [9]:
await test_collection.query(
    query_texts=["A cat Cat."], 
    include=["documents", "metadatas", "distances"], 
    n_results=2,
    )

2024-08-24 16:53:08,415 | INFO : Generating embeddings for 1 sentences with 1024 dimensions.
Batches: 100%|██████████| 1/1 [00:00<00:00,  1.59it/s]
2024-08-24 16:53:09,127 | INFO : HTTP Request: POST http://localhost:8011/api/v1/collections/f8c69627-b541-497c-9ba5-c48f01e31e59/query "HTTP/1.1 200 OK"


{'ids': [['1', '2']],
 'distances': [[0.09579048710135762, 0.6074025954511143]],
 'embeddings': None,
 'metadatas': [[{'id': 1}, {'id': 2}]],
 'documents': [['A nice Cat', 'An amazing Truck']],
 'uris': None,
 'data': None,
 'included': ['documents', 'metadatas', 'distances']}

# Add Data to Vector Database

In [16]:
from pathlib import Path

In [14]:
vector_db_client = await chromadb.AsyncHttpClient(host="localhost", port=8010)
commodity_code_collection = await vector_db_client.get_or_create_collection(
    "commodity_codes", 
    embedding_function=MixedbreadEmbeddingFunction(
        embedder=MixedbreadEmbedder(EMB_MODEL_PATH)
    ),
    metadata={
    "hnsw:space": "cosine",
    "hnsw:construction_ef": 100,
    }
)

2024-08-18 16:25:03,329 | INFO : Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-08-18 16:25:03,332 | INFO : Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.
2024-08-18 16:25:03,349 | INFO : HTTP Request: GET http://localhost:8010/api/v1/tenants/default_tenant "HTTP/1.1 200 OK"
2024-08-18 16:25:03,360 | INFO : HTTP Request: GET http://localhost:8010/api/v1/databases/default_database?tenant=default_tenant "HTTP/1.1 200 OK"
2024-08-18 16:25:03,363 | INFO : Use pytorch device_name: mps
2024-08-18 16:25:03,363 | INFO : Load pretrained SentenceTransformer: ../models/embeddings/mxbai-embed-large-v1
2024-08-18 16:25:06,225 | INFO : Model loaded from ../models/embeddings/mxbai-embed-large-v1
2024-08-18 16:25:06,279 | INFO : HTTP Request: POST http://localhost:8010/api/v1/collections?tenant=default_tenant&database=default_database "HTTP/1.1 200 OK"


In [22]:
DATA_PATH = Path().absolute().parent.joinpath("data")
COMMODITY_CODE_FILE_PATH = DATA_PATH.joinpath("raw", "commodity_codes.csv")

In [23]:
cc_df = pl.read_csv(COMMODITY_CODE_FILE_PATH, separator="|")

In [56]:
test_cc_list= CommodityCodesListChromaDB.from_dict(cc_df.to_dicts())
cc_documents = test_cc_list.get_chroma_input_document()

In [58]:
await commodity_code_collection.add(**cc_documents)

2024-08-18 18:49:04,202 | INFO : Generating embeddings for 66 sentences with 1024 dimensions.


Batches: 100%|██████████| 3/3 [00:02<00:00,  1.02it/s]
2024-08-18 18:49:07,193 | INFO : HTTP Request: GET http://localhost:8010/api/v1/pre-flight-checks "HTTP/1.1 200 OK"
2024-08-18 18:49:07,323 | INFO : HTTP Request: POST http://localhost:8010/api/v1/collections/c5eb333f-fd6f-40e1-8ab4-6dc873c773da/add "HTTP/1.1 201 Created"


In [59]:
await commodity_code_collection.count()

2024-08-18 18:49:30,595 | INFO : HTTP Request: GET http://localhost:8010/api/v1/collections/c5eb333f-fd6f-40e1-8ab4-6dc873c773da/count "HTTP/1.1 200 OK"


66

In [64]:
sample_result = await commodity_code_collection.query(
    query_texts=["I want to buy keyboard for computer."], 
    include=["documents"], 
    n_results=2,
    )

2024-08-18 19:11:49,314 | INFO : Generating embeddings for 1 sentences with 1024 dimensions.
Batches: 100%|██████████| 1/1 [00:00<00:00, 10.12it/s]
2024-08-18 19:11:49,444 | INFO : HTTP Request: POST http://localhost:8010/api/v1/collections/c5eb333f-fd6f-40e1-8ab4-6dc873c773da/query "HTTP/1.1 200 OK"


In [65]:
sample_result

{'ids': [['201010', '201030']],
 'distances': None,
 'embeddings': None,
 'metadatas': None,
 'documents': [['Desktop Computers for office workstations',
   'Monitors and Display Screens for computing']],
 'uris': None,
 'data': None,
 'included': ['documents']}